# Veri Aktarma

In [ ]:
# =======================
# 0. Paketler
# =======================
import os
import shutil
import numpy as np
import pandas as pd
import torch
from datasets import Dataset, Features, Sequence, Value
from transformers import (
    AutoTokenizer, AutoConfig, AutoModelForTokenClassification,
    TrainingArguments, Trainer, EarlyStoppingCallback
)
import evaluate
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

# =======================
# 1. Verileri oku (Train + Validation)
# =======================
train_df = pd.read_json("realistic_train.jsonl", lines=True)
val_df   = pd.read_json("realistic_val.jsonl", lines=True)

# Entities sütununu string yap -> Arrow hatasını önlemek için
for df in [train_df, val_df]:
    df["entities"] = df["entities"].apply(lambda x: [list(map(str, ent)) for ent in x])

# Dataset features tanımla
features = Features({
    "text": Value("string"),
    "entities": Sequence(Sequence(Value("string")))  # [[start, end, label], ...]
})

train_ds = Dataset.from_pandas(train_df, features=features)
val_ds   = Dataset.from_pandas(val_df, features=features)



# TOKENİZE ETME

In [2]:
# =======================
# 2. Tokenizer hazırla
# =======================
model_name = "dbmdz/bert-base-turkish-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Label listesi -> entity tiplerinden BIO formatı
unique_labels = set()
for ents in train_df["entities"]:
    for e in ents:
        unique_labels.add(e[2])   # sadece label kısmı

label_list = sorted(list(unique_labels))
label_list = ["O"] + [f"B-{l}" for l in label_list] + [f"I-{l}" for l in label_list]
label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for i, l in enumerate(label_list)}

# =======================
# 3. Label hizalama fonksiyonu (BIO)
# =======================
MAX_LEN = 128  # istersen 256 yapabilirsin (uzun metinlerde daha az truncation)

def tokenize_and_align_labels(examples):
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN,
        return_offsets_mapping=True
    )

    labels = []
    for i, offsets in enumerate(tokenized["offset_mapping"]):
        entity_list = examples["entities"][i]
        # JSONL içinden string gelen değerleri int'e çeviriyoruz
        ents = [(int(start), int(end), label) for start, end, label in entity_list]

        label_ids = []
        for start, end in offsets:
            if start == end:   # özel token (CLS/SEP vs.)
                label_ids.append(-100)
            else:
                tag = "O"
                for ent_start, ent_end, ent_label in ents:
                    if start >= ent_start and end <= ent_end:
                        tag = f"B-{ent_label}" if start == ent_start else f"I-{ent_label}"
                        break
                label_ids.append(label2id[tag])
        labels.append(label_ids)

    tokenized["labels"] = labels
    tokenized.pop("offset_mapping")
    return tokenized

train_tokenized = train_ds.map(tokenize_and_align_labels, batched=True)
val_tokenized   = val_ds.map(tokenize_and_align_labels, batched=True)



c:\Users\90532\anaconda3\envs\yolov5-env\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|██████████| 500/500 [00:00<00:00, 3596.40 examples/s]


# Model ve Metrik Tanımlama

In [3]:
# =======================
# 4. model_init (dropout'lar trial'dan)
# =======================
def model_init(trial=None):
    hidden_dropout = trial.suggest_float("hidden_dropout_prob", 0.0, 0.3) if trial else 0.1
    attn_dropout   = trial.suggest_float("attention_probs_dropout_prob", 0.0, 0.3) if trial else 0.1

    cfg = AutoConfig.from_pretrained(
        model_name,
        num_labels=len(label_list),
        id2label=id2label,
        label2id=label2id,
        problem_type="token_classification",
        hidden_dropout_prob=hidden_dropout,
        attention_probs_dropout_prob=attn_dropout,
    )
    return AutoModelForTokenClassification.from_pretrained(model_name, config=cfg)

# =======================
# 5. Değerlendirme metriği
# =======================
metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [
        [id2label[l] for l in label if l != -100]
        for label in labels
    ]
    true_preds = [
        [id2label[pred] for (pred, l) in zip(pred_row, label) if l != -100]
        for pred_row, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_preds, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }



# Train Hazırlığı

In [ ]:
# =======================
# 6. Argümanlar + HP Search Trainer
# =======================
hp_output_dir = "./_tmp_hp"      # deneme checkpoint'leri (sonunda silinecek)
best_output_dir = "./_tmp_best"  # en iyi HParam ile eğitimin checkpoint'leri (sonunda silinecek)
final_dir = "./bert-turkish-ner-final/yeni_veri3"  # EN İYİ model buraya

# GPU bf16 destekliyorsa otomatik aç
supports_bf16 = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8

base_args = TrainingArguments(
    output_dir=hp_output_dir,
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=200,                  # veri boyutuna göre ayarlayın (50–500)
    save_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_total_limit=1,

    # bu değerler hp_space tarafından override edilecek
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    warmup_ratio=0.0,
    lr_scheduler_type="linear",
    gradient_accumulation_steps=1,
    max_grad_norm=1.0,
    label_smoothing_factor=0.0,

    gradient_checkpointing=True,
    fp16=False,
    bf16=bool(supports_bf16),
    no_cuda=False,
    optim="adamw_8bit",              # bitsandbytes yoksa "adamw_torch"
    group_by_length=True,
    dataloader_num_workers=2,
    logging_dir="./logs",
    logging_steps=50,
    report_to="none",
    seed=42
)

trainer = Trainer(
    model_init=model_init,  # >>> önemli: model_init
    args=base_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=2e-4)]
)



c:\Users\90532\anaconda3\envs\yolov5-env\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Optimizasyon

In [5]:
# =======================
# 7. Optuna arama uzayı (genişletilmiş)
# =======================
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 4, 12),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8]),
        "gradient_accumulation_steps": trial.suggest_categorical("gradient_accumulation_steps", [1, 2, 4]),
        "weight_decay": trial.suggest_float("weight_decay", 1e-5, 1e-1, log=True),
        "warmup_ratio": trial.suggest_float("warmup_ratio", 0.0, 0.3),
        "lr_scheduler_type": trial.suggest_categorical(
            "lr_scheduler_type", ["linear", "cosine", "cosine_with_restarts", "polynomial"]
        ),
        "max_grad_norm": trial.suggest_float("max_grad_norm", 0.5, 1.0),
        "label_smoothing_factor": trial.suggest_float("label_smoothing_factor", 0.0, 0.1),

        # model_init içinde okunacak dropout'lar:
        "hidden_dropout_prob": trial.suggest_float("hidden_dropout_prob", 0.0, 0.3),
        "attention_probs_dropout_prob": trial.suggest_float("attention_probs_dropout_prob", 0.0, 0.3),
    }

def compute_objective(metrics):
    return metrics["eval_f1"]



# Optimizasyon Çalıştırma (OPTUNA)

In [6]:
# =======================
# 8. Hyperparameter Search (validation F1'i maksimize)
# =======================
os.makedirs(hp_output_dir, exist_ok=True)

best_trial = trainer.hyperparameter_search(
    direction="maximize",
    hp_space=hp_space,
    compute_objective=compute_objective,
    n_trials=40,                             # bütçenize göre 80-100 yapabilirsiniz
    backend="optuna",
    sampler=TPESampler(seed=42, multivariate=True),
    pruner=MedianPruner(n_startup_trials=10, n_warmup_steps=0),
)
print("Best trial:", best_trial)
print("Best hyperparameters:", best_trial.hyperparameters)

# =======================
# 9. En iyi hiperparametrelerle yeniden eğit
# =======================
os.makedirs(best_output_dir, exist_ok=True)

best_args = TrainingArguments(
    output_dir=best_output_dir,
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=200,
    save_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_total_limit=2,

    learning_rate=best_trial.hyperparameters["learning_rate"],
    per_device_train_batch_size=best_trial.hyperparameters["per_device_train_batch_size"],
    per_device_eval_batch_size=best_trial.hyperparameters["per_device_train_batch_size"],
    num_train_epochs=best_trial.hyperparameters["num_train_epochs"],
    weight_decay=best_trial.hyperparameters["weight_decay"],
    warmup_ratio=best_trial.hyperparameters["warmup_ratio"],
    lr_scheduler_type=best_trial.hyperparameters["lr_scheduler_type"],
    gradient_accumulation_steps=best_trial.hyperparameters.get("gradient_accumulation_steps", 1),
    max_grad_norm=best_trial.hyperparameters.get("max_grad_norm", 1.0),
    label_smoothing_factor=best_trial.hyperparameters.get("label_smoothing_factor", 0.0),

    gradient_checkpointing=True,
    fp16=False,
    bf16=bool(supports_bf16),
    no_cuda=False,
    optim="adamw_8bit",
    group_by_length=True,
    dataloader_num_workers=2,
    logging_dir="./logs",
    logging_steps=50,
    report_to="none",
    seed=42
)

best_trainer = Trainer(
    model_init=model_init,  # >>> yine model_init
    args=best_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=2e-4)]
)






c:\Users\90532\anaconda3\envs\yolov5-env\lib\site-packages\optuna\_experimental.py:32: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-08-21 13:50:07,542] A new study created in memory with name: no-name-091889de-63df-4856-abf3-d3dfdc222526
Trying to set hidden_dropout_prob in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set attention_probs_dropout_prob in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/13500 [00:00<?, ?it/s]c:\Users\90532\anaconda3\envs\yolov5-env\lib\site-packages\torch\_dyn

{'loss': 3.702, 'grad_norm': 8.353860855102539, 'learning_rate': 8.887988134073672e-07, 'epoch': 0.04}


  1%|          | 102/13500 [00:22<20:30, 10.88it/s]

{'loss': 3.1448, 'grad_norm': 7.468728065490723, 'learning_rate': 1.7775976268147343e-06, 'epoch': 0.09}


  1%|          | 152/13500 [00:26<20:46, 10.71it/s]

...

{'eval_loss': 7.976683264132589e-05, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_accuracy': 1.0, 'eval_runtime': 16.4075, 'eval_samples_per_second': 30.474, 'eval_steps_per_second': 7.618, 'epoch': 9.24}


 84%|████████▍ | 2600/3091 [17:56<03:23,  2.41it/s]
[I 2025-08-21 15:07:32,485] Trial 7 finished with value: 1.0 and parameters: {'learning_rate': 8.24757533344303e-05, 'num_train_epochs': 11, 'per_device_train_batch_size': 4, 'gradient_accumulation_steps': 4, 'weight_decay': 0.018709365688887352, 'warmup_ratio': 0.258219174976903, 'lr_scheduler_type': 'cosine', 'max_grad_norm': 0.5599326836668415, 'label_smoothing_factor': 0.033761517140362796, 'hidden_dropout_prob': 0.28287291117375574, 'attention_probs_dropout_prob': 0.09696087960622657}. Best is trial 6 with value: 1.0.
Trying to set hidden_dropout_prob in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set attention_probs_dropout_prob in the hyperparameter search but there is no corresponding field in `TrainingArguments`.


{'train_runtime': 1076.6842, 'train_samples_per_second': 45.974, 'train_steps_per_second': 2.871, 'train_loss': 0.1110292919866669, 'epoch': 9.24}


Some weights of BertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[W 2025-08-21 15:07:33,985] Trial 8 failed with parameters: {'learning_rate': 7.610438922351127e-05, 'num_train_epochs': 10, 'per_device_train_batch_size': 8, 'gradient_accumulation_steps': 1, 'weight_decay': 0.0001597766235483386, 'warmup_ratio': 0.08545214831324029, 'lr_scheduler_type': 'cosine', 'max_grad_norm': 0.6393232321183058, 'label_smoothing_factor': 0.09082658859666537, 'hidden_dropout_prob': 0.07186856720009173, 'attention_probs_dropout_prob': 0.04346846162736693} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\90532\anaconda3\envs\yolov5-env\lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_

KeyboardInterrupt: 

# En İyi Parametrelerle Eğitim

In [ ]:
train_result = best_trainer.train()
print(train_result)

# En iyi Modeli Kaydetme

In [ ]:
os.makedirs(final_dir, exist_ok=True)
best_trainer.save_model(final_dir)   # model + config
tokenizer.save_pretrained(final_dir)

# İsteğe bağlı: Geçici klasörleri temizle
for p in [hp_output_dir, best_output_dir]:
    try:
        shutil.rmtree(p, ignore_errors=True)
    except Exception as e:
        print(f"Temizlik hatası ({p}):", e)

print("Final model saved to:", final_dir)

# Model Çıktılarını İnceleme

In [ ]:
from transformers import pipeline
from html import escape
from IPython.display import HTML, display

# --- NER pipeline (GPU varsa device=0 ver)
ckpt = "./bert-turkish-ner-final/yeni_veri3/"
ner = pipeline(
    "token-classification",
    model=ckpt,
    tokenizer=ckpt,
    aggregation_strategy="simple",
    device=0  # GPU yoksa -1
)

# --- Test metni
text = (
    "Müşteri: “Merhaba, dün 15.03.2025 tarihinde hesabımdan "
    "TR12 3456 7890 1234 5678 9012 34 IBAN’ına 7.850,00 TL havale yapılmış gözüküyor. "
    "Benim bilgim dışında gerçekleşmiş, acil iptal edilmesini istiyorum. "
    "Ayrıca TCKN 12345678901 üzerinden yaptığım başvuruda da halen dönüş olmadı. "
    "Telefonum 0532 456 78 90, mail adresim ayse.kaya@example.com. "
    "Bu işlem Ziraat Teknoloji A.Ş. hesabına mı yapılmış, bilgi rica ederim.”"
)

# --- Tüm entity'ler için renk paleti
colors = {
    "PERSON":   "#4E79A7",  # mavi
    "ORG":      "#F28E2B",  # turuncu
    "DATE":     "#59A14F",  # yeşil
    "TIME":     "#FF33A6",  # pembe
    "IBAN":     "#E15759",  # kırmızı
    "AMOUNT":   "#EDC948",  # sarı
    "CURRENCY": "#B07AA1",  # mor
    "TCKN":     "#76B7B2",  # teal
    "CARD_NO":  "#9C755F",  # kahverengi
    "PLATE":    "#FF9DA7",  # açık pembe
    "ACCOUNT_NO": "#33B2FF",  # açık mavi
    "ADDRESS":  "#8CD17D",  # açık yeşil
    "LANDLINE": "#F39C12",  # turuncu-sarı
    "EMAIL":    "#7F7F7F",  # gri
    "PHONE":    "#17BECF",  # cyan
}

# --- HTML render fonksiyonu
def render_ner_html(text, preds, palette):
    preds = sorted(preds, key=lambda x: x["start"])
    out = []
    last = 0
    for p in preds:
        s, e, lab = p["start"], p["end"], p["entity_group"]
        out.append(escape(text[last:s]))
        color = palette.get(lab, "#FFD54F")
        out.append(
            f"<span style='background:{color}33;"
            f"border-bottom:2px solid {color}; border-radius:3px;"
            f"padding:0 2px;' title='{lab}'>"
            f"{escape(text[s:e])}"
            f"</span>"
        )
        last = e
    out.append(escape(text[last:]))

    # Lejant
    legend = "".join(
        f"<span style='background:{c}33;border-bottom:2px solid {c};"
        f"border-radius:3px;padding:2px 6px;margin:3px;display:inline-block'>{lbl}</span>"
        for lbl, c in palette.items()
    )

    html = (
        "<div style='font-family:system-ui,-apple-system,Segoe UI,Roboto,Arial;"
        "line-height:1.8;font-size:16px;white-space:pre-wrap;'>"
        + "".join(out) +
        "</div>"
        "<div style='margin-top:10px'>" + legend + "</div>"
    )
    return HTML(html)

# --- Model tahminleri ve gösterim
preds = ner(text)
display(render_ner_html(text, preds, colors))
